These files contain metadata for all 45,000 movies listed in the Full MovieLens Dataset. The dataset consists of movies released on or before July 2017. Data points include cast, crew, plot keywords, budget, revenue, posters, release dates, languages, production companies, countries, TMDB vote counts and vote averages.

This dataset also has files containing 26 million ratings from 270,000 users for all 45,000 movies. Ratings are on a scale of 1-5 and have been obtained from the official GroupLens website.

In [1]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Defining the selected columns and the function to extract genres

selected_columns = ['budget', 'genres', 'original_language', 'original_title',
                    'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime']

# Selecting the desired columns
movies_metadata_df = pd.read_csv("/content/movies_metadata.csv", low_memory=False)
movies_metadata_df = movies_metadata_df[selected_columns]

In [ ]:
# Defining the helper function to extract genres
def extract_genres(genres_str):
    try:
        genres_list = ast.literal_eval(genres_str)
        primary_genre = genres_list[0]['name'] if len(genres_list) > 0 else None
        secondary_genre = genres_list[1]['name'] if len(genres_list) > 1 else None
        tertiary_genre = genres_list[2]['name'] if len(genres_list) > 2 else None
        return primary_genre, secondary_genre, tertiary_genre
    except (ValueError, SyntaxError):
        return None, None, None

# Applying the function to extract genres
movies_metadata_df[['primary_genre', 'secondary_genre', 'tertiary_genre']] = movies_metadata_df['genres'].apply(
    lambda x: pd.Series(extract_genres(x)))

# Dropping the original 'genres' column
movies_metadata_df = movies_metadata_df.drop(columns=['genres'])

# Previewing the dataset after handling 'genres' column
movies_metadata_df.head()

In [ ]:
# Defining the helper function to handle unexpected values and extract production companies information
def extract_production_companies(prod_comp_str):
    try:
        prod_comp_list = ast.literal_eval(prod_comp_str)
        if not prod_comp_list or not isinstance(prod_comp_list, list):
            return None, 0
        primary_prod_comp = prod_comp_list[0]['name'] if len(prod_comp_list) > 0 else None
        num_prod_companies = len(prod_comp_list)
        return primary_prod_comp, num_prod_companies
    except (ValueError, SyntaxError):
        return None, 0

# Applying the function to extract production companies information
movies_metadata_df[['primary_production_company', 'num_production_companies']] = movies_metadata_df['production_companies'].apply(
    lambda x: pd.Series(extract_production_companies(x)))

# Defining the helper function to handle unexpected values and extract production countries information
def extract_production_countries(prod_country_str):
    try:
        prod_country_list = ast.literal_eval(prod_country_str)
        if not prod_country_list or not isinstance(prod_country_list, list):
            return None, 0
        primary_prod_country = prod_country_list[0]['iso_3166_1'] if len(prod_country_list) > 0 else None
        num_prod_countries = len(prod_country_list)
        return primary_prod_country, num_prod_countries
    except (ValueError, SyntaxError):
        return None, 0

# Applying the function to extract production countries information
movies_metadata_df[['primary_production_country', 'num_production_countries']] = movies_metadata_df['production_countries'].apply(
    lambda x: pd.Series(extract_production_countries(x)))

# Dropping the original 'production_companies' and 'production_countries' columns
movies_metadata_df = movies_metadata_df.drop(columns=['production_companies', 'production_countries'])

# Previewing the dataset after handling 'production_companies' and 'production_countries' columns
movies_metadata_df.head()

cleaning steps

In [ ]:
# Converting 'budget' to numeric
movies_metadata_df['budget'] = pd.to_numeric(movies_metadata_df['budget'], errors='coerce')

# Converting 'release_date' to datetime
movies_metadata_df['release_date'] = pd.to_datetime(movies_metadata_df['release_date'], errors='coerce')

# Extracting 'year' and 'month' from 'release_date'
movies_metadata_df['release_year'] = movies_metadata_df['release_date'].dt.year
movies_metadata_df['release_month'] = movies_metadata_df['release_date'].dt.month

# Dropping the original 'release_date' column
movies_metadata_df = movies_metadata_df.drop(columns=['release_date'])

# Handling missing values by filling them with appropriate placeholders or zeros
movies_metadata_df['primary_genre'].fillna('Unknown', inplace=True)
movies_metadata_df['secondary_genre'].fillna('Unknown', inplace=True)
movies_metadata_df['tertiary_genre'].fillna('Unknown', inplace=True)
movies_metadata_df['primary_production_company'].fillna('Unknown', inplace=True)
movies_metadata_df['primary_production_country'].fillna('Unknown', inplace=True)
movies_metadata_df[['budget', 'revenue', 'runtime', 'num_production_companies', 'num_production_countries', 'release_year', 'release_month']] = movies_metadata_df[['budget', 'revenue', 'runtime', 'num_production_companies', 'num_production_countries', 'release_year', 'release_month']].fillna(0)

# Previewing the cleaned dataset
movies_metadata_df.head()

In [ ]:
# Dropping the rows with budget = 0
movies_metadata_df = movies_metadata_df[movies_metadata_df['budget'] != 0].reset_index(drop=True)

# Checking the shape of the dataset after dropping rows with budget = 0
print(movies_metadata_df.shape)

# Setting 'original_title' as the index
movies_metadata_df.set_index('original_title', inplace=True)

Exploratory Data Analysis

In [ ]:
sns.set(style="whitegrid")

# 1. Distribution of Numerical Features
fig, ax = plt.subplots(figsize=(7, 5)) # Change axes to ax
sns.histplot(data=movies_metadata_df, x='budget', bins=30, ax=ax, kde=True, color='blue') # Use ax instead of axes[0, 0]
ax.set_title('Distribution of Budget') # Use ax instead of axes[0, 0]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

# Create a single subplot
fig, ax = plt.subplots(figsize=(7, 5))

# Plot on the single subplot
sns.histplot(data=movies_metadata_df, x='revenue', bins=30, ax=ax, kde=True, color='green')
ax.set_title('Distribution of Revenue') # Set the title for this subplot

plt.show() # Display the plot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

# Create a figure and a single axes object
fig, ax = plt.subplots(figsize=(7, 5))

# Use 'ax' to refer to this single subplot
sns.histplot(data=movies_metadata_df, x='runtime', bins=30, ax=ax, kde=True, color='red')
ax.set_title('Distribution of Runtime')

plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

# إنشاء قطعة فرعية
fig, ax = plt.subplots(figsize=(7, 5))

# رسم التمثيل البياني
sns.histplot(data=movies_metadata_df, x='num_production_companies', bins=30, ax=ax, kde=True, color='purple')

# تعيين العنوان
ax.set_title('Distribution of Number of Production Companies')

plt.show()

In [ ]:
# 2. Categorical Feature Analysis

# Number of Movies per Primary Genre
plt.figure(figsize=(8, 6))
sns.countplot(data=movies_metadata_df, y='primary_genre', order=movies_metadata_df['primary_genre'].value_counts().index, palette="viridis")
plt.title('Number of Movies per Primary Genre')
plt.xlabel('Number of Movies')
plt.ylabel('Primary Genre')
plt.show()

Primary Genre: Drama, Comedy, and Action are the top three genres with the highest number of movies. There's also a significant number of movies with unknown primary genres.

In [ ]:
# Number of Movies per Primary Production Country (Top 20)
plt.figure(figsize=(8, 6))

# Reset index before plotting to avoid duplicate index issues
sns.countplot(data=movies_metadata_df.reset_index(),
              y='primary_production_country',
              order=movies_metadata_df['primary_production_country'].value_counts().head(20).index,
              palette="viridis")

plt.title('Number of Movies per Primary Production Country (Top 20)')
plt.xlabel('Number of Movies')
plt.ylabel('Primary Production Country')
plt.show()


Primary Production Country: The United States (US) is by far the leading country in terms of movie production, followed by other countries like the United Kingdom (GB), France (FR), and Germany (DE).

In [ ]:
# Number of Movies Released Each Year
plt.figure(figsize=(18, 6))
sns.countplot(data=movies_metadata_df, x='release_year', palette="viridis")
plt.title('Number of Movies Released Each Year')
plt.xlabel('Release Year')
plt.ylabel('Number of Movies')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Monthly Distribution of Movie Releases
plt.figure(figsize=(12, 6))
sns.countplot(data=movies_metadata_df, x='release_month', palette="viridis")
plt.title('Monthly Distribution of Movie Releases')
plt.xlabel('Release Month')
plt.ylabel('Number of Movies')
plt.xticks(ticks=range(12), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.show()

In [ ]:
# 4. Correlation Analysis

# Calculating the correlation matrix
correlation_matrix = movies_metadata_df[['budget', 'revenue', 'runtime', 'num_production_companies', 'num_production_countries', 'release_year', 'release_month']].corr()

# Plotting the heatmap for correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix of Numerical Features')
plt.show()

تُظهر الخريطة الحرارية أعلاه الارتباط بين العديد من السمات الرقمية:

الميزانية والإيرادات: هناك ارتباط إيجابي بنسبة 0.74 بين الميزانية والإيرادات، مما يشير إلى أن الأفلام ذات الميزانيات الأعلى تميل إلى توليد إيرادات أعلى.
وقت التشغيل والإيرادات: الارتباط بين وقت التشغيل والإيرادات منخفض نسبيًا بنسبة 0.17.
عدد شركات الإنتاج والإيرادات: هذا أيضًا له ارتباط منخفض بنسبة 0.15 مع الإيرادات.
سنة الإصدار والإيرادات: هناك أيضًا ارتباط منخفض بنسبة 0.05 بين سنة الإصدار والإيرادات.



In [ ]:
# 5. Revenue Analysis

# Revenue vs Budget
plt.figure(figsize=(10, 6))
sns.scatterplot(data=movies_metadata_df, x='budget', y='revenue', color='blue', alpha=0.6)
plt.title('Revenue vs Budget')
plt.xlabel('Budget')
plt.ylabel('Revenue')
plt.show()

Revenue vs. Budget:

There is a clear trend indicating that movies with higher budgets generally tend to generate higher revenues.
There are also instances of movies with low budgets generating high revenues, indicating the presence of some successful low-budget movies.

In [ ]:
# Revenue vs Runtime
plt.figure(figsize=(10, 6))
sns.scatterplot(data=movies_metadata_df, x='runtime', y='revenue', color='green', alpha=0.6)
plt.title('Revenue vs Runtime')
plt.xlabel('Runtime')
plt.ylabel('Revenue')
plt.show()

Revenue vs. Runtime:

There is no distinct trend observed between revenue and runtime.

 ما متوسط الميزانية للأفلام في البيانات؟


---



In [ ]:
average_budget = movies_metadata_df['budget'].mean()
print(f"متوسط الميزانية: {average_budget}")

ما أكثر نوع من الأفلام (Primary Genre) تم إنتاجه؟

In [ ]:
most_common_genre = movies_metadata_df['primary_genre'].value_counts().idxmax()
print(f"أكثر نوع من الأفلام: {most_common_genre}")


ما هي السنة التي تم إنتاج أكبر عدد من الأفلام فيها؟

In [ ]:
most_movies_year = movies_metadata_df['release_year'].value_counts().idxmax()
print(f"السنة التي تم إنتاج أكبر عدد من الأفلام: {most_movies_year}")


ما إجمالي الإيرادات (Revenue) لجميع الأفلام؟



In [ ]:
total_revenue = movies_metadata_df['revenue'].sum()
print(f"إجمالي الإيرادات: {total_revenue}")


ما متوسط مدة الأفلام (Runtime) لكل نوع رئيسي؟

In [ ]:
average_runtime_by_genre = movies_metadata_df.groupby('primary_genre')['runtime'].mean()
print(average_runtime_by_genre)


ما الشركات التي أنتجت أكبر عدد من الأفلام؟

In [ ]:
top_production_companies = movies_metadata_df['primary_production_company'].value_counts().head(5)
print("أهم شركات الإنتاج وعدد الأفلام:")
print(top_production_companies)


الإيرادات حسب النوع الرئيسي للأفلام (Primary Genre)

In [ ]:
plt.figure(figsize=(12, 8))
genre_revenue = movies_metadata_df.groupby('primary_genre')['revenue'].mean().sort_values(ascending=False)
sns.barplot(x=genre_revenue.index, y=genre_revenue.values)
plt.title('Average Revenue by Primary Genre')
plt.xlabel('Primary Genre')
plt.ylabel('Average Revenue')
plt.xticks(rotation=45)
plt.show()


عدد الأفلام حسب النوع الرئيسي (Top 10 Genres)

In [ ]:
plt.figure(figsize=(12, 8))
top_genres_count = movies_metadata_df['primary_genre'].value_counts().head(10)
sns.barplot(x=top_genres_count.values, y=top_genres_count.index)
plt.title('Number of Movies by Primary Genre (Top 10 Genres)')
plt.xlabel('Number of Movies')
plt.ylabel('Primary Genre')
plt.show()


In [ ]:
print(movies_metadata_df['budget'])

In [ ]:
zero_budget_count = (movies_metadata_df['budget'] == 0).sum()
print(f"Number of movies with a budget of 0: {zero_budget_count}")


In [ ]:
france_movies_count = movies_metadata_df[movies_metadata_df['primary_production_country'] == 'FR'].shape[0]
print(f"The number of movies produced in France is: {france_movies_count}")
